In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision.datasets import MNIST
import torch.nn.functional as F
import wandb

In [7]:
torch.cuda.is_available()
torch.cuda.device(0)

In [8]:
wandb.init(project="mnist-model-evaluation", entity="kimkj")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kimkj. Use `wandb login --relogin` to force relogin


In [9]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)  # input channel, output channels, kernel size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [12]:
wandb.watch(net, log="all")


[]

In [13]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # log every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            wandb.log({"loss": running_loss / 100})
            running_loss = 0.0

[1, 100] loss: 0.6003665025532245
[1, 200] loss: 0.16417040321975945
[1, 300] loss: 0.10776503842324019
[1, 400] loss: 0.0918088820995763
[1, 500] loss: 0.07934333146549762
[1, 600] loss: 0.06789646556600928
[1, 700] loss: 0.0646851144777611
[1, 800] loss: 0.0583748265914619
[1, 900] loss: 0.06918909749481827
[2, 100] loss: 0.04191107958322391
[2, 200] loss: 0.044960605646483603
[2, 300] loss: 0.043821357116103174
[2, 400] loss: 0.050211394735379146
[2, 500] loss: 0.03748950225766748
[2, 600] loss: 0.03802134282072075
[2, 700] loss: 0.04106943976599723
[2, 800] loss: 0.03857151072472334
[2, 900] loss: 0.04431947691831738
[3, 100] loss: 0.029230414869380182
[3, 200] loss: 0.02618358903157059
[3, 300] loss: 0.01968860053399112
[3, 400] loss: 0.029337564525194468
[3, 500] loss: 0.02723388690617867
[3, 600] loss: 0.030478376239771024
[3, 700] loss: 0.03256398255005479
[3, 800] loss: 0.02404143688210752
[3, 900] loss: 0.03823239745746832
[4, 100] loss: 0.018396471673040652
[4, 200] loss: 0.

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
wandb.log({"accuracy": 100 * correct / total})

Accuracy of the network on the 10000 test images: 99 %


In [15]:
wandb.finish()


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy,▁
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,99.24
loss,0.00525
